In [ ]:
from pynvml import *




In [ ]:
nvmlInit()
deviceCount = nvmlDeviceGetCount()
for i in range(deviceCount):
    handle = nvmlDeviceGetHandleByIndex(i)
    print("Device", i, ":", nvmlDeviceGetName(handle))
nvmlShutdown()

In [ ]:
import tensorflow as tf

print("Built with CUDA: ", tf.test.is_built_with_cuda())
print("Built with GPU Support: ", tf.test.is_built_with_gpu_support())
import torch

print("CUDA available: ", torch.cuda.is_available())
print("CUDA version: ", torch.version.cuda)


In [ ]:
from numba import jit, cuda
import numpy as np
# to measure exec time
from timeit import default_timer as timer   
import tensorflow as tf

In [ ]:

  
# normal function to run on cpu
@jit(target_backend='cpu')
def func(a,n,size=5000):                                
    for i in range(n):
        a = tf.random.uniform(shape=[size, size])     
  
# function optimized to run on gpu 
@jit(target_backend='cuda')                         
def func2(a,n,size=5000):
    for i in range(n):
        a = tf.random.uniform(shape=[size, size])
if __name__=="__main__":
    n = 100                       
    a = np.ones(n, dtype = np.float64)
      
    start = timer()
    func(a,n)
    print("without GPU:", timer()-start)    
      
    start = timer()
    func2(a,n)
    print("with GPU:", timer()-start)

In [ ]:
import tensorflow as tf
import time

# Check if a GPU is available and if not, use a CPU
device_name = tf.test.gpu_device_name() if tf.test.is_gpu_available() else "/device:CPU:0"
print(device_name)
# Number of rounds for the test
rounds = 100
size = 5000

# Create some large matrix
a = tf.random.uniform(shape=[size, size])

# Run the test
start = time.time()

with tf.device(device_name):
    for _ in range(rounds):
        tf.linalg.matmul(a, a)

end = time.time()

print(f"Time taken to perform {rounds} rounds of matrix multiplication of size {size}x{size} : {end - start} seconds")


In [1]:


path='/teamspace/studios/this_studio/mistral-7b-gptq'
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model_name_or_path = "TheBloke/Mistral-7B-Instruct-v0.1-GPTQ"
# To use a different branch, change revision
# For example: revision="gptq-4bit-32g-actorder_True"
model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
                                             device_map="auto",
                                             trust_remote_code=False,
                                             revision="main")

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

prompt = "Tell me about AI"
prompt_template=f'''<s>[INST] {prompt} [/INST]
'''

print("\n\n*** Generate:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=512)
print(tokenizer.decode(output[0]))

# Inference can also be done using transformers' pipeline

# print("*** Pipeline:")
# pipe = pipeline(
#     "text-generation",
#     model=model,
#     tokenizer=tokenizer,
#     max_new_tokens=512,
#     do_sample=True,
#     temperature=0.7,
#     top_p=0.95,
#     top_k=40,
#     repetition_penalty=1.1
# )

# print(pipe(prompt_template)[0]['generated_text'])



config.json:   0%|          | 0.00/963 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.16G [00:00<?, ?B/s]

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/transformers/modeling_utils.py:4371: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(


generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.




*** Generate:
<s><s>[INST] Tell me about AI [/INST]
AI, or artificial intelligence, is the development of computer systems that are able to perform tasks that typically require human intelligence, such as visual perception, speech recognition, decision-making, and language translation. AI technology is constantly evolving and is being used in a wide range of applications, from virtual assistants and chatbots to self-driving cars and medical diagnosis. Some of the most common types of AI include narrow or weak AI, which is designed to perform a specific task, and general or strong AI, which is designed to be able to learn and perform any intellectual task that a human can.</s>


In [2]:
def get_response(input_text):
    prompt_template=f'''<s>[INST] {input_text} [/INST]
    '''
    input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
    output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=512)
    return tokenizer.decode(output[0])

In [1]:
from PyPDF2 import PdfReader
def get_pages(file_path):
    reader = PdfReader(file_path)
    res=[]
    for page in reader.pages:
        res.append(page.extract_text())
    return res


In [2]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /teamspace/studios/this_studio/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [7]:
from nltk.tokenize import sent_tokenize, word_tokenize
pages=get_pages("/teamspace/studios/this_studio/data_science/some tests/qlora.pdf")
pages[1]
len(pages)    
    


26

In [12]:
def num_of_tokens(text):
    return len(word_tokenize(text))
all_tokens=0
first_q_pages_text=''
for page in range(0,len(pages)//4):
    first_q_pages_text+=pages[page]
    all_tokens+=num_of_tokens(pages[page])
first_q_pages_text

'QL ORA: Efficient Finetuning of Quantized LLMs\nTim Dettmers∗Artidoro Pagnoni∗Ari Holtzman\nLuke Zettlemoyer\nUniversity of Washington\n{dettmers,artidoro,ahai,lsz}@cs.washington.edu\nAbstract\nWe present QLORA, an efficient finetuning approach that reduces memory us-\nage enough to finetune a 65B parameter model on a single 48GB GPU while\npreserving full 16-bit finetuning task performance. QLORAbackpropagates gradi-\nents through a frozen, 4-bit quantized pretrained language model into Low Rank\nAdapters (LoRA). Our best model family, which we name Guanaco , outperforms\nall previous openly released models on the Vicuna benchmark, reaching 99.3%\nof the performance level of ChatGPT while only requiring 24 hours of finetuning\non a single GPU. QLORAintroduces a number of innovations to save memory\nwithout sacrificing performance: (a) 4-bit NormalFloat (NF4), a new data type that\nis information theoretically optimal for normally distributed weights (b) Double\nQuantization to reduce